In [1]:
# CONECTAR CON DRIVE
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import sys
path ='/content/drive/MyDrive/cod/LEA3_Marketing'
os.chdir(path) ## volver la carpeta de repositorio directorio de trabajo
sys.path.append(path) ## agregarla al path, para leer archivos propios como paquetes

In [3]:
import numpy as np
import pandas as pd
import sqlite3 as sql
from sklearn.preprocessing import MinMaxScaler
from ipywidgets import interact ## para análisis interactivo
from sklearn import neighbors ### basado en contenido un solo producto consumido
import joblib
#### conectar_base_de_Datos
#!pip install ipywidgets

# CREAR CONEXIÓN CON LA BASE DE DATOS db_movies
con = sql.connect('data/db_movies')

# CREAR EL CURSOR
cur = con.cursor() ## se crea el cursor, que es el otro tipo de conexión para ejecutar las consultas

In [4]:
cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cur.fetchall())

[('ratings',), ('movies',)]


In [ ]:
# Películas mejor calificadas con al menos 30 valoraciones
pd.read_sql("""    SELECT m.title,
           AVG(r.rating) AS avg_rating,
           COUNT(*) AS rating_count
    FROM ratings r
    JOIN movies m ON r.movieId = m.movieId
    WHERE r.rating > 0
    GROUP BY m.title
    HAVING rating_count > 30
    ORDER BY avg_rating DESC
    LIMIT 10
""", con)




In [ ]:
# Películas más calificadas con promedio de calificación
pd.read_sql("""
    SELECT m.title,
           AVG(CASE WHEN r.rating = 0 THEN NULL ELSE r.rating END) AS avg_rating,
           COUNT(*) AS rating_count
    FROM ratings r
    JOIN movies m ON r.movieId = m.movieId
    GROUP BY m.title
    ORDER BY rating_count DESC
    LIMIT 20
""", con)

In [ ]:
## Pendiente de revisar ya sea mejor almacenando la consulta en un dataframe para manipular

#### los mejores calificados por año publicacion ###
pd.read_sql("""
  SELECT
      SUBSTR(m.title, LENGTH(m.title) - 5, 6) AS movie_year,  -- Extrae las últimas 4 cifras (año)
      SUBSTR(m.title, 1, LENGTH(m.title) - 7) AS movie_name,  -- Extrae el nombre quitando el año
      MAX(r.rating) AS best_rating
  FROM movies m
  JOIN ratings r ON m.movieId = r.movieId
  GROUP BY movie_year
  ORDER BY movie_year;"""
    ,con)

In [9]:
movies=pd.read_sql("""select * from movies""", con)


In [10]:
sep=movies['title'].str.split('(')

year=sep.str[1].str.replace(')','')
title=sep.str[0]

In [11]:
movies['year'] = movies['title'].str.extract(r'\((\d{4})\)$')
movies['title_clean'] = movies['title'].str.replace(r'\s*\(\d{4}\)$', '', regex=True)

In [12]:
movies.drop(columns=['title'], inplace=True)
movies.rename(columns={'title_clean': 'title'}, inplace=True)

In [ ]:
movies

In [ ]:
movies=movies[movies.columns[1:].tolist() + [movies.columns[0]]]
movies.drop(columns=['genres'], inplace=True)
movies

In [ ]:
#######################################################################
######## 2.1 Sistema de recomendación basado en contenido un solo producto - Manual ########
#######################################################################

movies_2=pd.read_sql('select * from movies_final', con )

movies_2.info()
movies_2['año_estreno']=movies_2.año_estreno.astype('int')
movies_2.info()

##### escalar para que año esté en el mismo rango ###

sc=MinMaxScaler()
movies_2[["year_sc"]]=sc.fit_transform(movies_2[['year_pub']])



## eliminar filas que no se van a utilizar ###

movies_2_dum1=movies_2.drop(columns=['isbn','i_url','year_pub','book_title']) ## ingresar las columnas a quitar ###

#### convertir a dummies

movies_2_dum1['book_author'].nunique()
movies_2_dum1['publisher'].nunique()

col_dum=['book_author','publisher']
movies_2_dum2=pd.get_dummies(movies_2_dum1,columns=col_dum)
movies_2_dum2.shape

joblib.dump(movies_2_dum2,"salidas\\movies_2_dum2.joblib") ### para utilizar en segundos modelos


